## IBM Capstone project
### Module Applied Data Science Capstone
#### This notebook will be mainly used for the capstone project



## Import libraries

In [191]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


## Question 1: creating a Dataframe

### Preparing data for analysis

In [192]:
#Creating soup object
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")

In [193]:
#Creating Dataframe
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [194]:
#Dataframe check
display(df.head())
display(df.shape)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


(103, 3)

## Question 2: latitude and the longitude coordinates

In [195]:
#Create a dataframe from GeoSpatial Dataset
geospatial = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geospatial.columns = ['PostalCode', 'Latitude', 'Longitude']
display(geospatial.head())

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [196]:
#merge original Dataframe with GeoSpatial data
df_ll = df.merge(geospatial, how='left', on='PostalCode')

In [197]:
#Merge check
display(df_ll.head())

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Question 3: Explore and cluster the neighborhoods

In [198]:
#Create a data frame with borough names containing the word Toronto
df_explore = df_ll[df_ll['Borough'].str.contains('Toronto')]
display(df_explore.head())

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [199]:
df_explore.shape

(39, 5)

In [218]:
# create map of Toronto to visualize neighborhoods lacations
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_explore['Latitude'], df_explore['Longitude'], df_explore['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### A map image was inserted because the folium is not working as expected on GitHub.
![Toronto neighborhood locations](toronto_neighborhood.jpg)

In [201]:
#Get information about venues in the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    CLIENT_ID = input('Enter Client Id:')
    CLIENT_SECRET = input('Enter Client Secret:')
    VERSION = '20180605'
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [202]:
toronto_venues = getNearbyVenues(names=df_explore['Neighborhood'],
                                   latitudes=df_explore['Latitude'],
                                   longitudes=df_explore['Longitude']
                                  )

Enter Client Id:FZPPYJO4H4RKTWFPC5AE2IEKJCYYON0HOME2XCSXOR5C3QL3
Enter Client Secret:IE3SD2WDO1FHSRYKYMVRR2DLPNGL5TSY2U0CVZEBETRM2SQC
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway L

In [203]:
#Check information about number of venues in neighborhoods
display(toronto_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count())

,Venue
Neighborhood,
Berczy Park,59
"Brockton, Parkdale Village, Exhibition Place",25
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16
Central Bay Street,62
Christie,16
Church and Wellesley,75
"Commerce Court, Victoria Hotel",100
Davisville,35
Davisville North,9


In [204]:
display('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

'There are 234 uniques categories.'

In [205]:
#Get dummies for Dataframe and compute the weight ov venue category for each neighborhood
toronto_processing = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_processing['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_processing.columns[-1]] + list(toronto_processing.columns[:-1])
toronto_processing = toronto_processing[fixed_columns]

toronto_grouped = toronto_processing.groupby('Neighborhood').mean().reset_index()
display(toronto_grouped)

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.040000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016129,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000
4,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.026667,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.013333,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333
6,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
7,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [206]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [210]:
#Explore top 15 venue categories for each neighborhood
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

display(neighborhoods_venues_sorted)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Restaurant,Pharmacy,Beer Bar,Lounge,Park,Comfort Food Restaurant,Belgian Restaurant,Bistro,Creperie
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Yoga Studio,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Office,Nightclub,Convenience Store,Italian Restaurant,Intersection
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport Gate,Airport,Airport Food Court,Gluten-free Restaurant,Diner,Escape Room
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Dessert Shop,Modern European Restaurant,Juice Bar,Korean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Office,New American Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Candy Store,Coffee Shop,Creperie,Discount Store,Escape Room,Electronics Store,Eastern European Restaurant
5,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gay Bar,Dance Studio,Yoga Studio,Fast Food Restaurant,Health & Beauty Service,Ramen Restaurant,American Restaurant
6,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega,American Restaurant,Thai Restaurant,Beer Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Gastropub
7,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Fried Chicken Joint,Indoor Play Area,Indian Restaurant,Farmers Market,Diner,Pharmacy,Brewery
8,Davisville North,Gym / Fitness Center,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Department Store,Pizza Place,Park,Gym,Dumpling Restaurant,Donut Shop,Dessert Shop,Dog Run,Eastern European Restaurant,Distribution Center
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Grocery Store,Bar,Café,Supermarket,Bank,Music Venue,Brewery,Park,Middle Eastern Restaurant,Donut Shop,Discount Store,Doner Restaurant,Dog Run


In [211]:
#Cluster processing
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_explore

toronto_merged = toronto_merged.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), how='left',on='Neighborhood')

display(toronto_merged)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Café,...,Breakfast Spot,Theater,Gym / Fitness Center,Event Space,Restaurant,Performing Arts Venue,Hotel,Historic Site,Wine Shop,French Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,...,Cosmetics Shop,Japanese Restaurant,Fast Food Restaurant,Ramen Restaurant,Bookstore,Hotel,Pizza Place,Electronics Store,Burger Joint,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Gastropub,Cocktail Bar,...,Seafood Restaurant,Hotel,Restaurant,Gym,Park,Lingerie Store,Italian Restaurant,Clothing Store,Department Store,Beer Bar
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Asian Restaurant,Health Food Store,Pub,Trail,...,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant,Donut Shop,Department Store,Eastern European Restaurant,Electronics Store,Escape Room
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,...,Farmers Market,Restaurant,Pharmacy,Beer Bar,Lounge,Park,Comfort Food Restaurant,Belgian Restaurant,Bistro,Creperie
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,...,Bubble Tea Shop,Burger Joint,Dessert Shop,Modern European Restaurant,Juice Bar,Korean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Office,New American Restaurant
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Nightclub,...,Restaurant,Baby Store,Athletics & Sports,Candy Store,Coffee Shop,Creperie,Discount Store,Escape Room,Electronics Store,Eastern European Restaurant
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Thai Restaurant,...,Gym,Deli / Bodega,Burrito Place,Sushi Restaurant,Pizza Place,Concert Hall,Steakhouse,Hotel,Salad Place,Bookstore
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Grocery Store,Bar,...,Supermarket,Bank,Music Venue,Brewery,Park,Middle Eastern Restaurant,Donut Shop,Discount Store,Doner Restaurant,Dog Run
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,2,Park,Convenience Store,Metro Station,Wings Joint,...,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [219]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### A map image was inserted because the folium is not working as expected on GitHub.
![Toronto neighborhood locations](toronto_cluster.jpg)

In [220]:
from IPython.display import IFrame

IFrame(src='filename.html', width=700, height=600)